In [1]:
import pandas as pd
from os import listdir
import itertools

In [2]:
# Coletando nomes dos arquivo CSV presentes na pasta "path_to_dir"
path_to_dir = "data/BDTD_feb2025"
filenames = listdir(path_to_dir)
csv_files = [ path_to_dir + "/" + filename for filename in filenames if filename.endswith( ".csv" ) ]

In [3]:
# Concatenando todos os arquivos
data = pd.DataFrame()
for csv in csv_files:
    data = pd.concat([data, pd.read_csv(csv)])
data = data.drop_duplicates(subset=['Título'], keep='first').reset_index(drop=True)

In [4]:
# Quebrando assuntos em lista
data["Assuntos em português"] = data["Assuntos em português"].apply(lambda x: x.split("||"))
data["Assuntos em inglês"] = data["Assuntos em inglês"].apply(lambda x: x.split("||"))

In [5]:
# Limpando o nome dos autores
autors = data["Autor(a)"].apply(lambda x: x.split("||"))

new_autors = []
for n in range(len(autors)):
    new_autors.append(autors[n][0])
    if len(autors[n]) > 1:
        if autors[n][1].startswith("http://lattes.cnpq.br/"):
            data.loc[n, "ID Lattes do(a) autor(a)"] = autors[n][1]

data["Autor(a)"] = new_autors

In [6]:
# Limpando os nomes dos orientadores

orientadores = data["Orientadores"].apply(lambda x: x.split("||"))

for n in range(len(orientadores)):
    if "Não informado pela instituição" in orientadores[n]:
        orientadores[n] = list(filter(lambda a: a != "Não informado pela instituição", orientadores[n]))

data["Orientadores"] = orientadores

In [7]:
# Limpando os nomes dos membros da banca

banca = data["Membros da banca"].apply(lambda x: x.split("||"))

for n in range(len(banca)):
    if "Não informado pela instituição" in banca[n]:
        banca[n] = list(filter(lambda a: a != "Não informado pela instituição", banca[n]))

data["Membros da banca"] = banca

In [8]:
# Limpando os textos do Resumo
data['Resumo'] = data['Resumo'].astype(str)
data.loc[data['Resumo'].str.contains('Orientador:'), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Orientadora:'), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Orientadores:'), 'Resumo'] = ''
data.loc[data['Resumo'] == '--', 'Resumo'] = ''
data.loc[data['Resumo'] == '.', 'Resumo'] = ''
data.loc[data['Resumo'].str.contains(r'Dissertação \(mestrado\) -', regex=True), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('também disponível em formato'), 'Resumo'] = ''
data.loc[data['Resumo'] == '[sem resumo]', 'Resumo'] = ''
data.loc[data['Resumo'] == 'abstract', 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Tese \(doutorado\)'), 'Resumo'] = ''
data.loc[data['Resumo'] == 'Não informado pela instituição', 'Resumo'] = ''

Filtrar as áreas de conhecimento e assuntos indesejados

In [9]:
print("Total de documentos antes da limpeza: ", len(data))

Total de documentos antes da limpeza:  24240


In [10]:
# Criando um DataFrame com os assuntos e áreas do conhecimento preprocessados
AC_assunt = data[['Área do conhecimento CNPq', 'Assuntos em português', 'Assuntos em inglês']].copy()

AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: x.split('||'))
AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: [a.split('::') if '::' in a else a for a in x])
AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: list(set([item.lower() for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])])))
AC_assunt.loc[:, 'Assuntos em português'] = AC_assunt['Assuntos em português'].apply(lambda x: [a.lower() for a in x])
AC_assunt.loc[:, 'Assuntos em inglês'] = AC_assunt['Assuntos em inglês'].apply(lambda x: [a.lower() for a in x])

In [11]:
# Funcao para limpar a base de dados de acordo com a area do conhecimento e lista de excessoes
def cleanAreaAssunto(area, lista_excessoes):
    
  indices_to_remove =  AC_assunt[AC_assunt.apply(lambda x : area in x['Área do conhecimento CNPq'] and 
                                                all(elem not in x['Assuntos em português'] for elem in lista_excessoes),
                                                axis=1)].index
  # Remove the rows with the specified indices
  return data.drop(indices_to_remove)

In [12]:
area = 'medicina veterinária'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [13]:
area = 'agronomia'
lista_excessoes = ['terras indígenas', 'conhecimento local indígena', 'índios da américa do sul - alimentos - amazônia', 'affirmative action',
                   'feira de agricultura familiar', 'local knowledge', 'conflitos fundiários', 'conhecimento tradicional associado',
                   'memória', 'quilombolas peoples', 'comunidade quilombola de itacoã-miri - acará (pa)', 'quilombo', 'comunidades tradicionais populares',
                   'guarani m’bya', 'community quilombola','xavante', 'mulheres quilombolas', 'traditional popular communities', 'conhecimento ecológico tradicional',
                   'negro (raça)', 'índios tikúna - alimentos', 'quilombolas - amazônia - alimentos', 'comunidades tradicionais', 'quilombolas - ilha do marajó',
                   'quilombolas - mato grosso', 'povos quilombolas', 'quilombolas - marajó, ilha do (pa)', 'terra preta de índio', 'índios tukúna – amazônia']
data = cleanAreaAssunto(area, lista_excessoes)

In [14]:
area = 'química'
lista_excessoes = ['educação quilombola;', 'educação intercultural;']
data = cleanAreaAssunto(area, lista_excessoes)

In [15]:
area = 'quimica'
lista_excessoes = ['movimento negro', 'racismo', 'mulheres negras nas exatas', 'lei 10.639', 'lei 10.639/03', 
                   'racismo epistêmico', 'discriminação racial na educação química']
data = cleanAreaAssunto(area, lista_excessoes)

In [16]:
area = 'bioquimica'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [17]:
area = 'bioquímica'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [18]:
area = 'engenharia química'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [19]:
area = 'engenharia quimica'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [20]:
print("Total de documentos após a limpeza: ", len(data))

Total de documentos após a limpeza:  23833


In [21]:
# Funcao para limpar a base de dados de acordo com assunto e lista de excessoes
def cleanAssunto(assunt, lista_excessoes):
  
  indices_to_remove = data.loc[AC_assunt.apply(lambda x : (assunt in x['Assuntos em português'] or 
                                                            assunt in x['Assuntos em inglês']) and
                                                            all(elem not in x['Assuntos em português'] for elem in lista_excessoes) and
                                                            all(elem not in x['Assuntos em inglês'] for elem in lista_excessoes),
                                                            axis=1)].index

  # Remove the rows with the specified indices
  return data.drop(indices_to_remove)

In [22]:
assunt = 'buracos negros'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [23]:
assunt = 'buraco negro'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [24]:
assunt = 'buracos negros (astronomia)'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [25]:
assunt = 'black hole'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [26]:
assunt = 'índia'
lista_excessoes = ['Império português', 'África', 'União Ibérica', 'Moçambique', 'Umbanda']
data = cleanAssunto(assunt, lista_excessoes)

In [27]:
assunt = 'física'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [28]:
assunt = 'cnpq::ciencias exatas e da terra::fisica'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [29]:
assunt = 'black carbon'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [30]:
assunt = 'negro de fumo'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [31]:
assunt = 'química'
lista_excessoes = ['Índios Guarani', 'indigenous', 'Movimento negro', 'Índios', 'Racismo', 'História e Cultura', 'Povos indígenas',
                   'Contemporary slavery', 'Quilombola School Education', 'Africanos e afro-brasileiros',
                   'Diretrizes Curriculares Nacionais para a Educação Escolar Quilombola na Educação Básica', 'Lei 10.639', 'Escravidão Contemporânea',
                   'Quilombos', 'Escravidão', 'African and afro-Brazilian people', 'Indígenas']
data = cleanAssunto(assunt, lista_excessoes)

In [32]:
assunt = 'black pepper'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [33]:
assunt = 'pimenta-do-reino'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [34]:
assunt = 'acácia-negra'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [35]:
assunt = 'acácia negra'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [36]:
assunt = 'acacia-negra'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [37]:
assunt = 'medicina veterinária'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [38]:
assunt = 'veterinária'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [39]:
assunt = 'microbiologia'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [40]:
assunt = 'filmes finos'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [41]:
assunt = 'adsorção'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [42]:
assunt = 'antioxidantes'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [43]:
assunt = 'black liquor'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [44]:
assunt = 'black metal'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [45]:
assunt = 'metal'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [46]:
assunt = 'black wattle'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [47]:
assunt = 'antocianinas'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [48]:
assunt = 'amora-preta'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [49]:
assunt = 'atividade antioxidante'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [50]:
assunt = 'black mirror'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [51]:
assunt = 'estresse oxidativo'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [52]:
assunt = 'black-scholes'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [53]:
assunt = 'geoquímica'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [54]:
assunt = 'black spot'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [55]:
assunt = 'podridão negra'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [56]:
assunt = 'corantes'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [57]:
assunt = 'petróleo'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [58]:
assunt = 'eletroquímica'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [59]:
assunt = 'glycine max'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [60]:
assunt = 'compostos fenólicos'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [61]:
assunt = 'compostos bioativos'
lista_excessoes = ['Comunidades tradicionais']
data = cleanAssunto(assunt, lista_excessoes)

In [62]:
assunt = 'cnpq::ciencias biologicas::bioquimica'
lista_excessoes = ['Cultura indígena']
data = cleanAssunto(assunt, lista_excessoes)

In [63]:
assunt = 'modelagem matemática'
lista_excessoes = ['Gavião ethnic group/Ikólóéhj', 'interculturality', 'indigenous school education',
                   'educação escolar indígena', 'etnomatemática', 'etnia Gavião/Ikólóéhj', 'ethnomathematics']
data = cleanAssunto(assunt, lista_excessoes)

In [64]:
assunt = 'fosfeto de índio'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [65]:
assunt = 'rhizobium'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [66]:
assunt = 'fixação biológica de nitrogênio'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [67]:
assunt = 'acerola'
lista_excessoes = []
data = cleanAssunto(assunt, lista_excessoes)

In [68]:
print("Total de documentos após a limpeza: ", len(data))

Total de documentos após a limpeza:  23172


In [69]:
# Reset the index if needed
data = data.reset_index(drop=True)

# Salvando como csv
path_to_dir_output = "data"
data.to_csv(path_to_dir_output + "/BDTD_Consolidado_feb2025.csv", index=False)